In [68]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


# Main( )

#### Updating variables:

In [69]:
files = get_content(directory='../data/')
last_file_update = None

for file in files:
    if 'ufc_fighters_stats' in file:
        last_file_update = get_last_update(file_path = '../data/' + file)

print('Last update: ' + last_file_update)

Last update: 07-04-2023


In [70]:
events_list = 'http://ufcstats.com/statistics/events/completed?page=all'
events = get_links(url=events_list)

#### Extracting data to data frame:

In [72]:
data = []
column_names = ['result','fighter','opponent',
                'f_kd','o_kd','f_str','o_str','f_td','o_td','f_sub','o_sub',
                'weight_class','method','type','round','time','date','url'
               ]

start_time = time()

for url in tqdm(events):
    data.extend(get_event_data(event_link=url))

end_time = time()        
print('Process time = %f sec' %(end_time - start_time))     
df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

100%|████████████████████████████████████████████████████████████████████████████████| 640/640 [06:57<00:00,  1.53it/s]

Process time = 417.075666 sec


,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,url
6417,win,Frank Hamaker,Thaddeus Luster,0,0,2,0,1,0,3,0,Open Weight,SUB,Keylock,1,4:52,11-03-1994,http://ufcstats.com/event-details/a6a9ab5a824e...
6418,win,Johnny Rhodes,David Levicki,0,0,11,4,1,0,0,0,Open Weight,KO/TKO,Punches,1,12:13,11-03-1994,http://ufcstats.com/event-details/a6a9ab5a824e...
6419,win,Patrick Smith,Ray Wizard,0,0,1,1,0,0,1,0,Open Weight,SUB,Guillotine Choke,1,0:58,11-03-1994,http://ufcstats.com/event-details/a6a9ab5a824e...


#### Sort by date:

In [73]:
df['date'] = pd.to_datetime(df.date, format='%d-%m-%Y') 
df = df.sort_values(by='date', ascending=False).reset_index(drop=True)
df.tail(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,url
6417,win,Royce Gracie,Remco Pardoel,0,0,0,0,1,0,1,0,Open Weight,SUB,Gi Choke,1,1:31,1994-03-11,http://ufcstats.com/event-details/a6a9ab5a824e...
6418,win,Royce Gracie,Patrick Smith,0,0,4,1,1,0,0,0,Open Weight KO/TKO ...,KO/TKO,Punches,1,1:17,1994-03-11,http://ufcstats.com/event-details/a6a9ab5a824e...
6419,win,Patrick Smith,Ray Wizard,0,0,1,1,0,0,1,0,Open Weight,SUB,Guillotine Choke,1,0:58,1994-03-11,http://ufcstats.com/event-details/a6a9ab5a824e...


#### Fighters list that require update:

In [74]:
last_file_update = '01-01-2022' # date for testing

In [83]:
fighter_list = df[['fighter','url']][df.date > last_file_update]
opponent_list = df[['opponent','url']][df.date > last_file_update].rename(columns={'opponent':'fighter'})
full_list = pd.concat([fighter_list,opponent_list]).reset_index(drop=True)
full_list #need to get only uniques

,fighter,url
0,Cory Sandhagen,http://ufcstats.com/event-details/aec273fcb765...
1,Nate Landwehr,http://ufcstats.com/event-details/aec273fcb765...
2,Maycee Barber,http://ufcstats.com/event-details/aec273fcb765...
3,Albert Duraev,http://ufcstats.com/event-details/aec273fcb765...
4,Daniel Pineda,http://ufcstats.com/event-details/aec273fcb765...
...,...,...
1159,Joanderson Brito,http://ufcstats.com/event-details/f5585e675af7...
1160,Jennifer Maia,http://ufcstats.com/event-details/f5585e675af7...
1161,Rogerio Bontorin,http://ufcstats.com/event-details/f5585e675af7...
1162,Chase Sherman,http://ufcstats.com/event-details/f5585e675af7...


#### Saving file:

In [43]:
df.to_csv('../data/ufc_matches_stats.csv', index=0)
print('../data/ufc_matches_stats.csv saved.')

../data/ufc_matches_stats.csv saved.


# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>